In [1]:
%cd protein_bert

/Users/qiaochufeng/Documents/GitHub/DS596-Project/protein_bert


In [1]:
# Model's hyperparameters
BATCH_SIZE = 1
ACCUM_BATCH = 32
MAX_PROTEIN_STR_LEN = 512
ENCODER_LR = 5e-6
GENERAL_LR = 3e-05
N_FROZEN_EPOCH = 1
GRADIENT_CHECKPOINT = False # True if RAM < 16GB
SEED = 43

# File's hyperparameters
FILE_DIR = "/protein_benchmarks/fluorescence.train.csv" 
COL_NAMES = ['seq','label'] # optional, but no need

# Trainer's hyperparameters
CPU_WORKERS = 2
N_CHECKPOINTS = 1 # number of top-k models you want to save
MONITOR_METRIC = "val_loss"
MONITOR_MODE = "min"
MIN_EPOCHS = 1
MAX_EPOCHS = 7 # Increase to 10-20 epochs to maximize accuracy (needs more GPU hours on Kaggle)
PATIENCE = MAX_EPOCHS
NUM_GPU = 0
PRECISION = 32
AMP_BACKEND = "native"

wandb_flag=True # Need a free wandb.ai account (retrieve a personal key there)
wandb_name="Run 4. %d-maxlen" % MAX_PROTEIN_STR_LEN
wandb_project="ProtBERT DeepLoc"

print(wandb_name)

Run 4. 512-maxlen


In [3]:
!pip install protein-bert pyfastx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.9/722.9 kB 2.1 MB/s eta 0:00:00-:--:--

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import pyfastx
import pandas as pd

from tensorflow import keras

from sklearn.model_selection import train_test_split

from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

In [12]:
train_set_file_path = './protein_bert/protein_benchmarks/fluorescence.valid.csv'
train_set = pd.read_csv(train_set_file_path).dropna().drop_duplicates()
train_set.head()

,seq,label
0,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGVGDATYGKLALKFI...,3.713898
1,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,3.577288
2,SKGEGLFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,3.677451
3,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,3.551528
4,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,3.718230


In [16]:
train_set, valid_set = train_test_split(train_set, test_size = 0.2, random_state = 0)
UNIQUE_LABELS = train_set['label'].unique()

In [19]:
OUTPUT_TYPE = OutputType(is_seq = False, output_type = 'categorical')
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

pretrained_model_generator, input_encoder = load_pretrained_model(
    local_model_dump_dir = './',
    local_model_dump_file_name = 'epoch_2_sample.pkl'
)

model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience = 1, factor = 0.25, min_lr = 1e-05, verbose = 1),
    keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True),
]

finetune(model_generator, input_encoder, OUTPUT_SPEC, train_set['seq'], train_set['label'], valid_set['seq'], valid_set['label'], \
        seq_len = 512, batch_size = 32, max_epochs_per_stage = 2, lr = 1e-04, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = 1e-05, callbacks = training_callbacks)

[2024_11_11-13:20:06] Training set: Filtered out 0 of 3088 (0.0%) records of lengths exceeding 510.
[2024_11_11-13:20:06] Validation set: Filtered out 0 of 772 (0.0%) records of lengths exceeding 510.


KeyError: 3.436640977859497